<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/C2_extract_random_frames_from_videos_for_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PROJECT = 'rc_car_tracking'
MODEL = 'yolo33'

COLAB_BASE_PROJECT = f'/content/{PROJECT}'

MAX_NUMBER_OF_FRAMES_PER_VIDEO = 100

#Google Drive settings
DRIVE_MOUNT_FOLDER = '/content/drive'
DRIVE_VIDEO_FOLDER = f'{DRIVE_MOUNT_FOLDER}/MyDrive/{PROJECT}/videos'
DRIVE_IMAGES_FOLDER = f'{DRIVE_MOUNT_FOLDER}/MyDrive/{PROJECT}/{MODEL}_images'

# Interval of frames considered in percentages (0 -> 0%, 1->100%)
INTERVAL_FRAMES = (0.2,0.8)

In [ ]:
import cv2
import os, datetime
import random
from os import listdir
import shutil
import time

In [ ]:
#mount Google Drive
from google.colab import drive
drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)
!ln -s $DRIVE_BASE_PROJECT $COLAB_BASE_PROJECT

In [ ]:
# Empty folder OUTPUT_IMAGES_FOLDER
shutil.rmtree(DRIVE_IMAGES_FOLDER, ignore_errors=True)
os.makedirs(DRIVE_IMAGES_FOLDER, exist_ok=True)

In [ ]:
# Extract random images in OUTPUT_IMAGES_FOLDER from each video in INPUT_VIDEOS_FOLDER
videos = os.listdir(DRIVE_VIDEO_FOLDER)

for i,video in enumerate(videos):
  
  if not video.endswith(".mp4"):
    continue;

  cap = cv2.VideoCapture(f"{DRIVE_VIDEO_FOLDER}/{video}")

  if not cap.isOpened(): 
    print(f'Could not open video {video}, {i+1}/{len(videos)}')
    break;

  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print(f'Processing video {video} with {length} frames, {i+1}/{len(videos)}')

  deck = list(range(int(length * INTERVAL_FRAMES[0]), int(length * INTERVAL_FRAMES[1])))

  if len(deck) < MAX_NUMBER_OF_FRAMES_PER_VIDEO:
    print(f'Video {video} does not contain enough frames ({length} total frames) to extract in according to MAX_NUMBER_OF_FRAMES_PER_VIDEO ({MAX_NUMBER_OF_FRAMES_PER_VIDEO}) set')
    break;

  random.shuffle(deck)  

  num_image = 0      
  counter = 0;

  while num_image < MAX_NUMBER_OF_FRAMES_PER_VIDEO:
    num_frame = deck.pop(0)
    cap.set(cv2.CAP_PROP_POS_FRAMES, num_frame - 1)
    success, image = cap.read()
    if success:
      index = video.split('_')[0]
      file_name = f'{num_frame}_{index}.jpg'
      path = os.path.join(DRIVE_IMAGES_FOLDER, file_name)
      cv2.imwrite(path, image)
      print(f"\rExtracting random frame {counter + 1} of {MAX_NUMBER_OF_FRAMES_PER_VIDEO}", end='', flush=True) 
      num_image += 1
      counter += 1
    else:
      print(f'Error in reading frame {num_frame}...')

  print('\n')